In [1]:
import pandas as pd
import numpy as np

import clickhouse_connect


client = clickhouse_connect.get_client(
    host='index.bostrom.bronbro.io', port=443,
    username='bronbro', password='spac3b0x')

def query_to_df(query: str) -> pd.DataFrame:
    query_res = client.query(query=query)
    return pd.DataFrame(query_res.result_rows, columns=query_res.column_names)

In [2]:
query_1 = r'''
SELECT *
FROM (SELECT signer,
             groupArrayIf(passport_nickname, passport_nickname != '')                        as passport_nicknames,
             uniqExactIf(proof_address, proof_address != '')                                 as proof_addresses,
             uniqExactIf(remove_address, remove_address != '')                               as remove_addresses,
             uniqExactIf(gift_claiming_address, gift_claiming_address != '')                 as claim_addresses,
             uniqExactIf(release_address, release_address != '')                             as release_addresses,
             round(sum(gift_amount_gboot), 1)                                                as claim_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, '0x')), 1)     as claim_eth_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'cosmos')), 1) as claim_cosmos_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'osmo')), 1)   as claim_osmosis_gboot,
             round(sumIf(gift_amount_gboot, startsWith(gift_claiming_address, 'terra')), 1)  as claim_terra_gboot,
             claim_gboot * 0.29                                                              as releasable_gboot,
             round(sum(released_gboot), 1)                                                   as release_gboot
      FROM (SELECT signer,
                   arrayJoin(JSONExtractArrayRaw(messages))                     as message_row,
                   trim(BOTH '"' FROM
                        replaceAll(replaceAll(message_row, '\\', ''), '@', '')) as message,
                   JSONExtractString(message, 'type')                           as message_type,
                   JSONExtractString(message, 'sender')                         as message_sender,
                   JSONExtractString(message, 'contract')                       as message_contract,
                   JSONExtractString(message, 'msg')                            as message_msg,
                   JSON_VALUE(message_msg, '$.create_passport.nickname')        as passport_nickname,
                   JSON_VALUE(message_msg, '$.proof_address.address')           as proof_address,
                   JSON_VALUE(message_msg, '$.remove_address.address')          as remove_address,
                   JSON_VALUE(message_msg, '$.release.gift_address')            as release_address,
                   round(toInt64OrZero(JSON_VALUE(message_msg, '$.claim.gift_amount')) / 1e9,
                         1)                                                     as gift_amount_raw_gboot,
                   JSON_VALUE(message_msg, '$.claim.gift_claiming_address')     as gift_claiming_address,
                   toInt64OrZero(replace(
                           JSON_VALUE(JSONExtractArrayRaw(raw_log)[1], '$.events[1].attributes[0].value'),
                           'boot', '')) /
                   1e9                                                          as released_gboot,
                   toInt64OrZero(JSON_VALUE(JSONExtractArrayRaw(raw_log)[1], '$.events[5].attributes[4].value')) /
                   1e9                                                          as gift_amount_gboot, date
            FROM (SELECT height,
                         signer,
                         raw_log,
                         messages,
                         date
                  FROM `spacebox`.transaction FINAL
                  LEFT ANY JOIN (
                    SELECT height,
                           toDate(timestamp) as date
                    FROM `spacebox`.block
                    PREWHERE block.height > 10800000
                  ) as b
                  ON transaction.height = b.height
                  PREWHERE transaction.success = true
                  and transaction.height > 10800000
                  )
            WHERE message_type == '/cosmwasm.wasm.v1.MsgExecuteContract'
              and message_contract in ('bostrom1xut80d09q0tgtch8p0z4k5f88d3uvt8cvtzm5h3tu3tsy4jk9xlsfzhxel',
                                       'bostrom16t6tucgcqdmegye6c9ltlkr237z8yfndmasrhvh7ucrfuqaev6xq7cpvek')
            ORDER BY height DESC)
      GROUP BY signer)
-- WHERE release_gboot > 0
ORDER BY release_gboot DESC
'''

gift_by_singer = query_to_df(query_1)

In [3]:
gift_by_singer

,signer,passport_nicknames,proof_addresses,remove_addresses,claim_addresses,release_addresses,claim_gboot,claim_eth_gboot,claim_cosmos_gboot,claim_osmosis_gboot,claim_terra_gboot,releasable_gboot,release_gboot
0,bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6,[etwet46w4],3640,3640,1549,1505,1582.8,1582.8,0.0,0.0,0.0,459.012,455.5
1,bostrom10un89xcvrtjjmsw8qlsednz5ymkp8dmt956wr8,[],0,0,0,8,0.0,0.0,0.0,0.0,0.0,0.000,443.5
2,bostrom1trt5q36aqdx6l2ca3r5hush2hm9vfu699j9ksn,[fbfbhbvcccccc],2147,2141,2142,2143,1020.5,1020.5,0.0,0.0,0.0,295.945,334.0
3,bostrom10cnk6upcn3u553m046e872gwsxkvy328fdkh4z,[gfsjyrjryhkyifulk],14,6,8,8,447.8,447.8,0.0,0.0,0.0,129.862,272.3
4,bostrom1g596jpnw3flmxd3hr3jdvvasylcz4yzaueww5p,[sfaerfwe11],8,2,6,6,328.4,328.4,0.0,0.0,0.0,95.236,248.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15585,bostrom1ednc63kwckcz6arhzm28f7undd4gqlvygya0ks,[ngnfn554],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15586,bostrom19jcktgz30hz8vd637cgcxmmqvrut3k0ky68fv3,[54545454545454545454545454542],2,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15587,bostrom153cw09k759z52lkmh4xxsgy37gtt6splyf8388,[jasvoyager],0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
15588,bostrom1gt4kqr6jsfd987qsgzj8wh7twg8es6ye50qglr,[laloaaaaaaaaa],3,0,0,0,0.0,0.0,0.0,0.0,0.0,0.000,0.0


In [4]:
gift_by_singer.groupby('release_addresses').agg({'release_gboot': sum, 'signer': np.count_nonzero, 'release_addresses': sum})

,release_gboot,signer,release_addresses
release_addresses,,,
0,0.0,7905,0
1,5422.9,4432,4432
2,4855.3,1503,3006
3,3578.0,543,1629
4,3664.5,282,1128
5,2329.9,182,910
6,1849.7,163,978
7,1290.8,123,861
8,4521.2,415,3320


In [5]:
gift_by_singer.sum()

signer                 bostrom1l74ggk5tsn7d2l5qv8fxk3zvdc7fpwrud8gnw6...
passport_nicknames     [etwet46w4, fbfbhbvcccccc, gfsjyrjryhkyifulk, ...
proof_addresses                                                    58809
remove_addresses                                                   21187
claim_addresses                                                    25138
release_addresses                                                  27065
claim_gboot                                                      97712.3
claim_eth_gboot                                                  86742.6
claim_cosmos_gboot                                                5981.4
claim_osmosis_gboot                                               4270.4
claim_terra_gboot                                                  700.9
releasable_gboot                                               28336.567
release_gboot                                                    29977.7
dtype: object